## Notebook principal
<br>
Implémentation du modèle (fonctions objectifs, contraintes et résolution/optimisation).

### Importations

In [1]:
import gurobipy as gp # solver
from gurobipy import GRB
import numpy as np

In [2]:
from InstanceClass import Instance # custom class built to load and encode .json files (instances)

### Loading the data

In [3]:
instance = Instance(path="data/large_instance.json")
instance.build_instance()

In [4]:
instance.variables # check that everything works as expected

{'NP': 25,
 'NC': 6,
 'NA': 10,
 'H': 36,
 'GAIN': [15,
  30,
  30,
  30,
  70,
  50,
  50,
  80,
  80,
  20,
  20,
  40,
  20,
  20,
  20,
  80,
  25,
  20,
  25,
  60,
  50,
  50,
  45,
  40,
  50],
 'PENALTIES': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          3.,  6.,  9., 12., 15., 18., 21., 24., 27., 30.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  6.,  9., 12.,
         15., 18., 21., 24., 27., 30., 33., 36., 39., 42.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  3.,  6.,  9., 12., 15., 18., 21., 24., 27., 30.,
         33., 36., 39., 42., 45., 48., 51., 54., 57., 60.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  3.,  6.,  9., 12., 15., 18., 21., 24.,
         27., 

### Création de la matrice des affectations
<br>
On définit la matrice A des affections des collaborateurs sur les projets (de taille n_c ∗ h  ∗ n_p ∗ n): 
<br>
$
𝐴 = {𝑎_{𝑖,𝑗,𝑘,l}}
$
<br>


On en extrait une matrice 𝐴𝑝 = {𝑎𝑝𝑗,𝑘} indiquant si à l’instant j le projet vient d’être
rendu 


In [5]:
def solution_creation(variables):
    shape=(variables["NC"],variables["H"],variables["NP"],variables["NA"])
    return shape

def calcul_work(Affectation):
    rep = [np.sum([Affectation[i][j] for i in range(3)],axis=0) for j in range(np.shape(Affectation)[1])]
    return np.array(rep)

def calcul_temporel_work(work,horizon,nb_projet):
    rep = [[gp.quicksum(work[0:i])[j] for i in range(1,horizon+1)] for j in range(nb_projet)]
    return np.array(rep)

def calcul_work_final(work):
    rep=gp.quicksum(work)  
    return rep


In [6]:
# données propres au problème que l'on va modéliser
shape = solution_creation(instance.variables)
GAIN = instance.variables["GAIN"]
PENALTIES = instance.variables["PENALTIES"]
COST_PROJECT = instance.variables["COST_PROJECT"]
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]
BENEFICES = np.array([GAIN[i]-PENALTIES[i] for i in range(L)])
MAX_COST = np.max(COST_PROJECT)

In [7]:
print(I,J,L,K)

6 36 25 10


### Instanciation du modèle

In [8]:
model = gp.Model("matrix1") # instanciation du modèle

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


### Définition des variables

In [9]:
# Create variables
Affectation = model.addMVar(shape=(I,J,L,K), vtype=GRB.BINARY, name="Affectation") # matrice personnes/jours/projet/compétence (tous les projets)
Done_Project = model.addMVar(shape=(L,J), vtype=GRB.BINARY, name="Done_Project") # matrice projets/temps (tous les projets)


In [10]:
z_fo2 = model.addVar(vtype=GRB.INTEGER,name="Max_Projet")
delta = model.addMVar(shape=(L,I),vtype=GRB.INTEGER,name="Contrainte2")

In [11]:
z_fo3 = model.addVar(vtype=GRB.INTEGER,name="Duree_Max_Projet")
Begin_Project = model.addMVar(shape=(L,J), vtype=GRB.BINARY, name="Begin_Project") # matrice projets/temps (tous les projets)
u = np.arange(J)+1

In [12]:
final_work = calcul_work_final(calcul_work(Affectation)) # somme des travaux menés par projet (sert à vérifier qu'un projet est complété)
# dimension de final_work : matrice projets/compétences (somme sur les employés par compétence et par projet, au temps final)
temp_work = calcul_temporel_work(calcul_work(Affectation),instance.variables["H"],L) # somme cumulée dans le temps du travail effectué par projet et 
# par compétence (on veut l'utiliser plutôt que final_work à terme) ; temp_work[-1] = finl_work

### Définition des fonctions objectifs

In [13]:
# définition des objectifs
model.setObjective(gp.quicksum(gp.quicksum(Done_Project*BENEFICES)), GRB.MAXIMIZE)
# model.setObjective(z_fo2, GRB.MINIMIZE)
# model.setObjective(z_fo3, GRB.MINIMIZE)


### Définition des contraintes

In [14]:
model.addConstrs((Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)), name="Qualifications_Constraint")
model.addConstrs((gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1]*K))) <= 1-CONGES[i][j] for i in range(I) for j in range(J) for l in range(L)), name="Day_off_Constraint") 
model.addConstrs((temp_work[i][j] >= COST_PROJECT[i]*Done_Project[i][j] for i in range(L) for j in range(J)), name="Done") 
model.addConstrs((gp.quicksum(Done_Project[i]) <= 1 for i in range(L)), name="Only_Done_One_Time") 

{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>,
 5: <MConstr () *awaiting model update*>,
 6: <MConstr () *awaiting model update*>,
 7: <MConstr () *awaiting model update*>,
 8: <MConstr () *awaiting model update*>,
 9: <MConstr () *awaiting model update*>,
 10: <MConstr () *awaiting model update*>,
 11: <MConstr () *awaiting model update*>,
 12: <MConstr () *awaiting model update*>,
 13: <MConstr () *awaiting model update*>,
 14: <MConstr () *awaiting model update*>,
 15: <MConstr () *awaiting model update*>,
 16: <MConstr () *awaiting model update*>,
 17: <MConstr () *awaiting model update*>,
 18: <MConstr () *awaiting model update*>,
 19: <MConstr () *awaiting model update*>,
 20: <MConstr () *awaiting model update*>,
 21: <MConstr () *awaiting model update*>,
 22: <MConstr () *awaiting model update*>,
 23: <MConstr () *awa

In [15]:
epsilon=0.0001

In [16]:
model.addConstrs((gp.quicksum(delta)[i] <= z_fo2 for i in range(I)), name="C2.1") 
model.addConstrs(((J*(1-delta[l][i]))<=gp.quicksum(gp.quicksum(Affectation[i])[l]) for i in range(I) for l in range(L)), name="C2.2")
model.addConstrs((gp.quicksum(gp.quicksum(Affectation[i])[l])+epsilon<=(J*delta[l][i]) for i in range(I) for l in range(L)), name="C2.3") 

{(0, 0): <MConstr () *awaiting model update*>,
 (0, 1): <MConstr () *awaiting model update*>,
 (0, 2): <MConstr () *awaiting model update*>,
 (0, 3): <MConstr () *awaiting model update*>,
 (0, 4): <MConstr () *awaiting model update*>,
 (0, 5): <MConstr () *awaiting model update*>,
 (0, 6): <MConstr () *awaiting model update*>,
 (0, 7): <MConstr () *awaiting model update*>,
 (0, 8): <MConstr () *awaiting model update*>,
 (0, 9): <MConstr () *awaiting model update*>,
 (0, 10): <MConstr () *awaiting model update*>,
 (0, 11): <MConstr () *awaiting model update*>,
 (0, 12): <MConstr () *awaiting model update*>,
 (0, 13): <MConstr () *awaiting model update*>,
 (0, 14): <MConstr () *awaiting model update*>,
 (0, 15): <MConstr () *awaiting model update*>,
 (0, 16): <MConstr () *awaiting model update*>,
 (0, 17): <MConstr () *awaiting model update*>,
 (0, 18): <MConstr () *awaiting model update*>,
 (0, 19): <MConstr () *awaiting model update*>,
 (0, 20): <MConstr () *awaiting model update*>,
 (

In [17]:
model.addConstrs((gp.quicksum(Begin_Project[i])<= 1 for i in range(L)), name="Only_Begin_One_Time") 
model.addConstrs((gp.quicksum(Begin_Project[i]) == gp.quicksum(Done_Project[i]) for i in range(L)), name="Begin_Done") 
model.addConstrs((gp.quicksum((Done_Project[i]-Begin_Project[i])*u)+1<=z_fo3 for i in range(L)), name="Duration") 
model.addConstrs((temp_work[i][j][k]/MAX_COST <= gp.quicksum(Begin_Project[i][0:(j+1)]) for i in range(L) for j in range(J) for k in range(K)), name="Cant_begin_without_work") 


{(0, 0, 0): <MConstr () *awaiting model update*>,
 (0, 0, 1): <MConstr () *awaiting model update*>,
 (0, 0, 2): <MConstr () *awaiting model update*>,
 (0, 0, 3): <MConstr () *awaiting model update*>,
 (0, 0, 4): <MConstr () *awaiting model update*>,
 (0, 0, 5): <MConstr () *awaiting model update*>,
 (0, 0, 6): <MConstr () *awaiting model update*>,
 (0, 0, 7): <MConstr () *awaiting model update*>,
 (0, 0, 8): <MConstr () *awaiting model update*>,
 (0, 0, 9): <MConstr () *awaiting model update*>,
 (0, 1, 0): <MConstr () *awaiting model update*>,
 (0, 1, 1): <MConstr () *awaiting model update*>,
 (0, 1, 2): <MConstr () *awaiting model update*>,
 (0, 1, 3): <MConstr () *awaiting model update*>,
 (0, 1, 4): <MConstr () *awaiting model update*>,
 (0, 1, 5): <MConstr () *awaiting model update*>,
 (0, 1, 6): <MConstr () *awaiting model update*>,
 (0, 1, 7): <MConstr () *awaiting model update*>,
 (0, 1, 8): <MConstr () *awaiting model update*>,
 (0, 1, 9): <MConstr () *awaiting model update*>,


In [18]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i3-6006U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 77806 rows, 55952 columns and 2685541 nonzeros
Model fingerprint: 0x9e05b106
Variable types: 0 continuous, 55952 integer (55800 binary)
Coefficient statistics:
  Matrix range     [1e-01, 4e+01]
  Objective range  [1e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 4e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 77053 rows and 54957 columns
Presolve time: 2.55s
Presolved: 753 rows, 995 columns, 13978 nonzeros
Variable types: 0 continuous, 995 integer (987 binary)

Root relaxation: objective 2.650000e+02, 159 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

### Vérification du respect des contraintes

In [19]:
Affectation_solution = Affectation.X
Done_Project_solution = Done_Project.X
Begin_Project_solution = Begin_Project.X
NB_Project_solution = z_fo2.X
Duration_solution = z_fo3.X

In [20]:
cumul_work = np.cumsum(np.sum(Affectation_solution,axis=0),axis=0)

In [21]:
# Qualifications
for i in range(I):
    for j in range(J):
        for l in range(L):
            for k in range(K):
                assert Affectation_solution[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] 

# One day One work
for i in range(I):
    for j in range(J):
        assert np.sum(Affectation_solution[i][j])<= 1- CONGES[i][j]
        
# Project Done one time
assert (np.sum(Done_Project_solution,axis=1)<=1).all()

# Assert Project done
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert(
                (cumul_work[
                    np.where(Done_Project_solution[l] == 1)[0][0]
                ][l] >= COST_PROJECT[l]).all()
            )
        
# Project Done & Begin
assert((
np.sum(Done_Project_solution,axis=1)==np.sum(Begin_Project_solution,axis=1)
).all())

# Project are begin before
assert((
Done_Project_solution<=np.cumsum(Begin_Project_solution,axis=1)
).all())

# Assert Duration Project
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert(
            (np.where(Done_Project_solution[l] == 1)[0][0] - np.where(Begin_Project_solution[l] == 1)[0][0])+1 <= Duration_solution
            )
        
# Assert Nb project worked
for i in range(i):
    assert np.sum(np.sum(np.sum(Affectation_solution,axis=1)[i],axis=1)>=1)<=NB_Project_solution
    
# Working imply Beginned
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert np.where(np.sum(cumul_work,axis=2).transpose()[l]>=1)[0][0] >= np.where(Begin_Project_solution[l]>=1)[0][0]

### Plot 

In [22]:
instance.load_solution(model,Affectation,Done_Project,Begin_Project,z_fo2,z_fo3)
instance.save_solution(path="medium_instance/")
instance.print_kpi()

The project Job1 is selected in the solution and done during the day 26 and beginned during the day 1.
The project Job2 is selected in the solution and done during the day 22 and beginned during the day 1.
The project Job3 is selected in the solution and done during the day 15 and beginned during the day 1.
The project Job4 is selected in the solution and done during the day 17 and beginned during the day 1.
The project Job5 is selected in the solution and done during the day 14 and beginned during the day 1.
The project Job6 is selected in the solution and done during the day 33 and beginned during the day 1.
The project Job7 isn't selected in the solution.
The project Job8 isn't selected in the solution.
The project Job9 isn't selected in the solution.
The project Job10 isn't selected in the solution.
The project Job11 isn't selected in the solution.
The project Job12 is selected in the solution and done during the day 29 and beginned during the day 1.
The project Job13 isn't selecte

In [23]:
np.sum(np.sum(np.sum(Affectation_solution,axis=1),axis=1)>=1)

8

In [24]:
x = np.transpose(np.sum(np.sum(Affectation_solution,axis=0),axis=2)>=1)

In [25]:
np.max(np.cumsum(x,axis=1))

12

In [26]:
# Toy
# quand on maximise le gain y
print("y1",65/80)
print("y2",3/5)
print("y3",3/5)


y1 0.8125
y2 0.6
y3 0.6


In [27]:
# Medium
print("y1",176/510)
print("y2",6/15)
print("y3",9/22)

y1 0.34509803921568627
y2 0.4
y3 0.4090909090909091


In [36]:
# Large
print("y1",253/1020)
print("y2",8/25)
print("y3",12/36)

y1 0.24803921568627452
y2 0.32
y3 0.3333333333333333


In [38]:
p1=(1,0.6,0.6)

In [39]:
p2=(0.1875,0,0)

In [41]:
p1

TypeError: tuple indices must be integers or slices, not tuple

In [52]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[p1[0], p1[0], p2[0], p2[0], p1[0], p1[0], p2[0], p2[0]],
        y=[p1[1], p2[1], p2[1], p1[1], p1[1], p2[1], p2[1], p1[1]],
        z=[p1[2], p1[2], p1[2], p1[2], p2[2], p2[2], p2[2], p2[2]],

        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='y',
        opacity=0.50
    )
])
fig.update_layout(
    scene = dict(
                xaxis = dict(nticks=10, range=[0,1],),
                yaxis = dict(nticks=10, range=[0,1],),
                zaxis = dict(nticks=10, range=[0,1],),
                xaxis_title='GAIN',
                yaxis_title='Maximum Project by worker',
                zaxis_title='Maximum duration project',
    ),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))
fig.show()